A notebook to run the entire pipeline using the ground-truth noise classifier during the graph-contruction phase

In [1]:
from functools import partial
from pathlib import Path

import torch

from gnn_tracking.training.tc import TCModule
from gnn_tracking.models.graph_construction import MLGraphConstructionFromChkpt
from gnn_tracking.models.track_condensation_networks import GraphTCNForMLGCPipeline
from gnn_tracking.metrics.losses.metric_learning import GraphConstructionHingeEmbeddingLoss
from gnn_tracking.postprocessing.dbscanscanner import DBSCANHyperParamScanner
from pytorch_lightning import Trainer
from gnn_tracking.utils.loading import TrackingDataModule
from gnn_tracking.training.callbacks import PrintValidationMetrics
from gnn_tracking.utils.versioning import assert_version_geq

from torch_geometric.data import Data
from torch import nn

assert_version_geq("23.12.0")

# Some change

# Configure the data

In [2]:
# data_dir = Path.cwd().resolve().parent.parent / "test-data" / "data" / "point_clouds" / "v8"
data_dir = Path("/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_1")
assert data_dir.is_dir()

In [3]:
dm = TrackingDataModule(
    train=dict(
        dirs=[data_dir],
        stop=1,
    ),
    val=dict(
        dirs=[data_dir],
        start=1,
        stop=2,
    ),
    identifier="point_clouds_v8"
)

In [4]:
model = GraphTCNForMLGCPipeline(
    node_indim=22,
    edge_indim=44,
    h_dim=192,
    e_dim=192,
    hidden_dim=192,
    h_outdim=24,
    L_hc=5,
    alpha_latent=0.5,
    n_embedding_coords=8
)

/scratch/gpfs/aj2239/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'hc_in' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['hc_in'])`.


In [5]:
preproc = MLGraphConstructionFromChkpt(
    ml_chkpt_path="~/gnn_tracking/tutorials/notebooks/lightning_logs/version_53977655/checkpoints/epoch=999-step=1000.ckpt",
    max_num_neighbors=10,
    max_radius=1,
    use_embedding_features=True,
    build_edge_features=True
)

[04:24:26] DEBUG: Getting class MLModule from module gnn_tracking.training.ml
[04:24:26] DEBUG: Loading checkpoint ~/gnn_tracking/tutorials/notebooks/lightning_logs/version_53977655/checkpoints/epoch=999-step=1000.ckpt
[04:24:26] DEBUG: Getting class GraphConstructionFCNN from module gnn_tracking.models.graph_construction
[04:24:26] DEBUG: Getting class NoiseClassifierModel from module gnn_tracking.models.graph_construction
[04:24:26] DEBUG: Getting class GraphConstructionHingeEmbeddingLoss from module gnn_tracking.metrics.losses.metric_learning
[04:24:27] DEBUG: Checkpoint loaded. Model ready to go.


In [6]:
loss_fct = GraphConstructionHingeEmbeddingLoss(
    lw_repulsive=0.05,
    pt_thld=0.9,
    max_num_neighbors=256,
    p_attr=2,
    p_rep=2,
    r_emb=1
)

In [7]:
cluster_scanner = DBSCANHyperParamScanner(
    n_trials=60,
    n_jobs=6,
    keep_best=30
)

In [8]:
lmodel = TCModule(
    model=model,
    preproc=preproc,
    loss_fct=loss_fct,
    cluster_scanner=cluster_scanner,
    optimizer=partial(torch.optim.Adam, lr=8*1e-4),
    scheduler=partial(torch.optim.lr_scheduler.LinearLR,
                                            start_factor=1,
                                            end_factor=0.1,
                                            total_iters=50)
)

# Training

In [9]:
trainer = Trainer(
    max_epochs=1000,
    accelerator="gpu",
    log_every_n_steps=1,
    callbacks=[PrintValidationMetrics()],
)
trainer.fit(model=lmodel, datamodule=dm)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100 80GB PCIe MIG 1g.10gb') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
[04:24:36] INFO: DataLoader will load 1 graphs (out of 900 available).
[04:24:36] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_1/data21000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_1/data21000_s0.pt
[04:24:36] INFO: DataLoader will load 1 graphs (out of 900 available).
[04:24:36] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_cl

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/scratch/gpfs/aj2239/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.
/scratch/gpfs/aj2239/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

[04:26:31] WARNING: WARNING: ran out of memory (OOM), skipping batch. If this happens frequently, decrease the batch size. Will abort if we get 10 consecutive OOM errors.
/scratch/gpfs/aj2239/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/loops/optimization/automatic.py:129: `training_step` returned `None`. If this was on purpose, ignore this warning...


...

Validation: |          | 0/? [00:00<?, ?it/s]

NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf fou

                    Validation epoch=9                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Metric                         ┃         Value ┃ Error ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━┩
│ attractive                     │       4.04002 │   nan │
│ attractive_weighted            │       4.04002 │   nan │
│ best_dbscan_eps                │       0.02085 │   nan │
│ best_dbscan_min_samples        │       1.00000 │   nan │
│ n_edges_att                    │   37293.00000 │   nan │
│ n_edges_rep                    │ 2588199.00000 │   nan │
│ n_hits_oi                      │   10284.00000 │   nan │
│ repulsive                      │     106.52496 │   nan │
│ repulsive_weighted             │       5.32625 │   nan │
│ total                          │       9.36627 │   nan │
│ trk.double_majority            │       0.00036 │   nan │
│ trk.double_majority_pt0.5      │       0.00052 │   nan │
│ trk.double_majority_pt0.9      │       0.00132 │   nan

[04:37:44] WARNING: WARNING: ran out of memory (OOM), skipping batch. If this happens frequently, decrease the batch size. Will abort if we get 10 consecutive OOM errors.


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "/home/aj2239/gnn_tracking/src/gnn_tracking/models/mlp.py", line 49, in forward
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
                ~~~~~ <--- HERE
        return x
  File "/scratch/gpfs/aj2239/micromamba/envs/gnn/lib/python3.11/site-packages/torch/nn/modules/activation.py", line 101, in forward
    def forward(self, input: Tensor) -> Tensor:
        return F.relu(input, inplace=self.inplace)
               ~~~~~~ <--- HERE
  File "/scratch/gpfs/aj2239/micromamba/envs/gnn/lib/python3.11/site-packages/torch/nn/functional.py", line 1471, in relu
        result = torch.relu_(input)
    else:
        result = torch.relu(input)
                 ~~~~~~~~~~ <--- HERE
    return result
RuntimeError: CUDA out of memory. Tried to allocate 486.00 MiB. GPU 0 has a total capacty of 9.50 GiB of which 484.94 MiB is free.